In [1]:
import pickle
import tensorflow as tf

from util import *
from biomedical_qa.inference.inference import Inferrer, get_model_and_session
from biomedical_qa.sampling.bioasq import BioAsqSampler

import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


In [2]:
CONFIG = "../model_checkpoints/simple_pointer_with_chars/bioasq_finetune_dr05_sigmoid_bioasqtrainer_ckptits200/config.pickle"

model, sess = get_model_and_session(CONFIG, ["cpu:0"])
inferrer = Inferrer(model, sess, beam_size=20)

Loading Model...
Using weights: ../model_checkpoints/simple_pointer_with_chars/bioasq_finetune_dr05_sigmoid_bioasqtrainer_ckptits200/final_model.tf
Restoring Weights...


In [3]:
vocab = inferrer.model.embedder.vocab
rev_vocab = [""] * len(vocab)
for w, i in vocab.items():
    rev_vocab[i] = w

In [4]:
sampler = BioAsqSampler("../data/BioASQ", ["dev.json"], 16, vocab=vocab, shuffle=False, types=["factoid"])

# Run Model

In [5]:
predictions = inferrer.get_predictions(sampler)

In [6]:
sorted_predictions = [predictions[key] for key in sorted(predictions.keys())]

In [7]:
def is_correct(prediction):
    answers = maybe_flatten_list(prediction.question.question_json["original_answers"])
    answers = [a.lower() for a in answers]
    return prediction.answer_strings[0].lower() in answers

In [8]:
incorrect_predictions = [prediction for prediction in sorted_predictions
                                    if not is_correct(prediction)]

# Analysis

In [9]:
# Answer is written differently, but 100% correct
SHOULD_COUNT_CORRECT = 0
# Answer is probably correct (according to my understanding) or very close to a correct answer
SOMEWHAT_CORRECT = 0
# Answer candidates were given in the question, but the system failed to output one of the options
MULTPLE_CHOICE = 0
# The correct answer is within the top 5
WITHIN_TOP5 = 0
# Answer type of the top answer is wrong
WRONG_ANSWER_TYPE = 0

In [10]:
print_prediction(incorrect_predictions[0], rev_vocab)

Id:
  51542e44d24251bc05000081
Question:
  mutations in which gene determine response to both erlotinib and gefitinib?
Answers:
 * epidermal growth factor receptor (EGFR) gene
Predicted Answers:
 * ('EGFR', 0.99854434)
 * ('NSCLC', 0.99184448)
 * ('EGFR-TKI', 0.97593504)
 * ('T790M EGFR', 0.88556099)
 * ('non-small cell lung cancer', 0.82213044)



**Comment**: Should count correct

In [11]:
SHOULD_COUNT_CORRECT += 1

In [12]:
print_prediction(incorrect_predictions[1], rev_vocab, with_context=True)

Id:
  517901bc8ed59a060a00003b
Question:
  the antibodies mk-3475 and ct-011 have shown promising results in treating malignancies. which protein are they targeting?
Answers:
 * PD-1
Predicted Answers:
 * ('Anti-PD1', 0.99995399)
 * ('nivolumab', 0.99620271)
 * ('anti-PD-1', 0.99125767)
 * ('PD-1', 0.76946491)
 * ('', 0.59915453)
we find that using an anti-PD-1 antibody (CT-011)
PD-1 blockade by CT-011, anti-PD-1 antibody,
. Presence of CT-011, an anti-PD1 antibody,
CT-011, a novel monoclonal anti-PD-1 antibody
CT-011, a humanized antibody interacting with PD-1,
Anti-PD1 (nivolumab and MK-3475)
anti-PD-1 antibodies MK-3475



**Comment**: Should have known that PD-1 is the protein

In [13]:
WRONG_ANSWER_TYPE += 1
WITHIN_TOP5 += 1

In [14]:
print_prediction(incorrect_predictions[2], rev_vocab)

Id:
  52bf1f2d03868f1b06000015
Question:
  which drug is benserazide usually co-administered with?
Answers:
 * L-Dopa
Predicted Answers:
 * ('levodopa', 0.97044557)
 * ('L-DOPA', 0.92194831)
 * ('Eticlopride', 0.89383131)
 * ('rats', 0.88922167)
 * ('Tolcapone', 0.78956878)



**Comment**: Should count correct

In [15]:
SHOULD_COUNT_CORRECT += 1

In [16]:
print_prediction(incorrect_predictions[3], rev_vocab, top_k=10, with_context=True)

Id:
  52bf217003868f1b0600001b
Question:
  what can nothobranchius furzeri be used as a model system for?
Answers:
 * aging research
Predicted Answers:
 * ('resveratrol', 0.99818003)
 * ('drug validation', 0.99495518)
 * ('aging', 0.92815971)
 * ('biological curiosity', 0.92063314)
 * ('Japanese Medaka', 0.87659377)
 * ('genetic control of life-history traits', 0.78218514)
 * ('aging research community', 0.71681327)
 * ('N', 0.71493572)
 * ('stickleback', 0.67471093)
 * ('comparative genomics of aging', 0.66058344)
The short-lived annual fish Nothobranchius furzeri shows extremely short captive life span and accelerated expression of age markers, making it an interesting model system to investigate the effects of experimental manipulations on longevity and age-related pathologies.
Owing to large differences in aging phenotypes in different lines, N. furzeri could represent a model system for studying the genetic control of life-history traits in natural populations.
It is very close to

**Comment**: #2 should count correct. Answer one is completely off (wrong type, wrong context).

In [17]:
WRONG_ANSWER_TYPE += 1
WITHIN_TOP5 += 1

In [18]:
print_prediction(incorrect_predictions[4], rev_vocab, with_context=True)

Id:
  52d63b2803868f1b0600003a
Question:
  from which tissue was the nci-h520 cell-line derived?
Answers:
 * Lung
Predicted Answers:
 * ('lung cancer', 0.98956686)
 * ('non-small cell lung cancer', 0.98268563)
 * ('human non-small cell lung cancer', 0.93219846)
 * ('cancer', 0.56639189)
 * ('lung cancer cells', 0.069223545)
on human non-small cell lung cancer cell lines, A549, NCI-H460 and NCI-H520.
NCI-H520 and NCI-H596 lung cancer cells. 



**Comment**: Confusing Context

In [19]:
WRONG_ANSWER_TYPE += 1

In [20]:
print_prediction(incorrect_predictions[5], rev_vocab, with_context=True)

Id:
  52fa6ac72059c6d71c000055
Question:
  is the transcriptional regulator bach1 an activator or a repressor?
Answers:
 * Repressor
Predicted Answers:
 * ('Bach1', 0.9980951)
 * ('transcriptional repressor', 0.955405)
 * ('repressor', 0.85322726)
 * ('Bach-1', 0.83765137)
 * ('Nrf2\nach1', 0.54562265)
the impact of BACH1 repression on transcription
Bach1 is a repressor of the oxidative stress response
transcriptional repressor Bach-1, 
Bach1, a transcriptional repressor of the HMOX1 gene
The mechanism underlying Bach1-mediated HO-1 repression is less well understood
Transcription factor BACH1 [BTB (broad-complex, tramtrack and bric-a-brac) and CNC (cap'n'collar protein) homology 1] binds to ARE-like sequences, functioning as a transcriptional repressor 
Bach1) is a transcriptional repressor o



**Comment**: Question clearly offers two choices, model fails to recognize this.

In [21]:
MULTPLE_CHOICE += 1
WRONG_ANSWER_TYPE += 1
WITHIN_TOP5 += 1

In [22]:
print_prediction(incorrect_predictions[6], rev_vocab, 10, with_context=True)

Id:
  530c7f52970c65fa6b000010
Question:
  oxantel is used for periodontitis treatment. how does it work?
Answers:
 * Oxantel disrupts polymicrobial biofilm
Predicted Answers:
 * ('inhibit fumarate reductase', 0.99005467)
 * ('inhibit P', 0.98471665)
 * ('fumarate reductase', 0.95816457)
 * ('oxantel', 0.88623297)
 * ('to inhibit fumarate reductase (Frd) activity in some pathogenic bacteria', 0.86248612)
 * ('planktonic cells', 0.84832942)
 * ('anthelmintic', 0.76686734)
 * ('significantly inhibited biofilm formation', 0.7591272)
 * ('polymicrobial biofilm', 0.75016397)
 * ('gingivalis homotypic biofilm formation', 0.75009876)
Oxantel disrupts polymicrobial biofilm development of periodontal pathogens
he anthelmintic drug oxantel has been shown to inhibit fumarate reductase (Frd) activity in some pathogenic bacteria and inhibit P. gingivalis homotypic biofilm formation.
Oxantel, a cholinergic anthelmintic and fumarate reductase inhibitor, significantly inhibited biofilm formation by P.

**Comment**: Not sure if top answer would be correct too. Generally, the answer type of some answers is clearly incorrect ("inhibit P", "oxantel")

In [23]:
SOMEWHAT_CORRECT += 1

In [24]:
print_prediction(incorrect_predictions[7], rev_vocab, with_context=True)

Id:
  530cf4fe960c95ad0c00000b
Question:
  which type of lung cancer is afatinib used for?
Answers:
 * EGFR-mutant non small cell lung carcinoma
 * EGFR-mutant NSCLC
Predicted Answers:
 * ('non-small cell lung cancer', 0.95893651)
 * ('non-small-cell lung cancer', 0.84804577)
 * ('gefitinib', 0.84352475)
 * ('metastatic non-small-cell lung cancer', 0.7865296)
 * ('HER2', 0.67783076)
Afatinib is a novel irreversible inhibitor of the ErbB family members EGFR, tyrosine kinase-type cell surface receptors HER2 and HER4. It shows preclinical efficacy in NSCLC with common EGFR-activating mutations and the T790M mutation typically associated with EGFR TKI resistance
At present, only first-generation EGFR-tyrosine kinase inhibitors (TKIs) (erlotinib and gefitinib) are available for clinical use. Second-generation irreversible EGFR-TKIs, such as afatinib, are still in clinical trials



**Comment**: Should count correct:

In [25]:
SHOULD_COUNT_CORRECT += 1

In [26]:
print_prediction(incorrect_predictions[8], rev_vocab, with_context=True)

Id:
  530cf54dab4de4de0c000009
Question:
  what is the reason for the narcolepsy cases developed after h1n1 influenza vaccination?
Answers:
 * The proposed mechanism for postvaccination narcolepsy is one in which an environmental trigger causes or enhances an antibody-mediated autoimmune response in patients with a preexisting genetic susceptibility.
Predicted Answers:
 * ('loss of hypothalamic hypocretin/orexin (hcrt)', 0.99192101)
 * ('support an autoimmune mechanism', 0.98578459)
 * ('molecular mimicry', 0.97660595)
 * ('autoimmune reaction', 0.97629696)
 * ('winter H1N1 infections', 0.96838886)
The proposed mechanism for postvaccination narcolepsy is one in which an environmental trigger causes or enhances an antibody-mediated autoimmune response in patients with a preexisting genetic susceptibility. 
The loss of hypothalamic hypocretin/orexin (hcrt) producing neurons causes narcolepsy with cataplexy. An autoimmune basis for the disease has long been suspected and recent results ha

**Comment**: Not sure if #1 should count as correct. "autoimmune reaction" should definitely be a valid answer.

In [27]:
SOMEWHAT_CORRECT += 1
WITHIN_TOP5 += 1

In [28]:
print_prediction(incorrect_predictions[9], rev_vocab, with_context=True, context_char_limit=10000)

Id:
  53130a77e3eabad02100000f
Question:
  which hormone deficiency is implicated in the costello syndrome ?
Answers:
 * Growth hormone
Predicted Answers:
 * ('adrenal insufficiency', 0.99540198)
 * ('Hypoglycemia', 0.99084151)
 * ('osteoporosis', 0.98533767)
 * ('RasGRF1', 0.98475939)
 * ('growth hormone deficiency', 0.93529189)
Measurements obtained after growth hormone exposure in 15 individuals were excluded in this analysis.
Furthermore, we observed that RasGRF1 becomes phosphorylated in ARMS after stimulation by several pro-metastatic factors, such as SDF-1 and HGF/SF, as well as after exposure to growth-promoting Igf-2 and insulin.
Progressively worsening hypertrophic cardiomyopathy in a child with newly diagnosed Costello syndrome while receiving growth hormone therapy.
This report highlights two important concepts: the association of genetic syndromes with hypertrophic cardiomyopathy and the possibility of worsening severity of hypertrophic cardiomyopathy linked to growth horm

**Comment**: Wrong Answer Type. Also "growth hormone" is the most common string close to "deficiency"...

In [29]:
WRONG_ANSWER_TYPE += 1
WITHIN_TOP5 += 1

In [30]:
print_prediction(incorrect_predictions[10], rev_vocab, with_context=True, context_char_limit=1500)

Id:
  53189b05b166e2b80600001e
Question:
  which glands are subject to attack by lymphocytes in sjogren's syndrome?
Answers:
 * The lacrimal and/or salivary glands
Predicted Answers:
 * ('salivary', 0.98992836)
 * ('xerophthalmia', 0.98288071)
 * ('lachrymal', 0.96502686)
 * ('exocrine glands', 0.95683044)
 * ('lacrimal', 0.95671654)
In Sjögren's syndrome the autoimmune response is directed against the exocrine glands, which, as histopathological hallmark of the disease, display persistent and progressive focal mononuclear cell infiltrates.
Sjögren's syndrome (SS) is an autoimmune disease characterized by clonal B cell attack of the exocrine glands and dysregulated expression of B cell-activating factor (BAFF).
Sjögren's syndrome (SjS) is a human autoimmune disease characterized by exocrine dysfunction resulting from chronic autoimmune attack primarily against the lacrimal and/or salivary glands.
Sjögren's syndrome is an autoimmune disease in which immune cells chronically attack the l

**Comment**: Should be a list question, two elements are #1 and #3. However, "xerophthalmia" is completely off (wrong answer type).

In [31]:
SOMEWHAT_CORRECT += 1
WITHIN_TOP5 += 1

In [32]:
print_prediction(incorrect_predictions[11], rev_vocab, with_context=True, context_char_limit=1000)

Id:
  5319ac99b166e2b806000034
Question:
  which is the third subunit of the tsc1-tsc2 complex upstream of mtorc1?
Answers:
 * TBC1D7
Predicted Answers:
 * ('TSC1-TSC2-TBC1D7', 0.99963701)
 * ('TSC-TBC', 0.99812037)
 * ('TBC1D7', 0.99184626)
 * ('Rheb', 0.97572714)
 * ('Rheb-GAP', 0.94920486)
The tuberous sclerosis complex (TSC) tumor suppressors form the TSC1-TSC2 complex, which limits cell growth in response to poor growth conditions. Through its GTPase-activating protein (GAP) activity toward Rheb, this complex inhibits the mechanistic target of rapamycin (mTOR) complex 1 (mTORC1), a key promoter of cell growth.
Here, we identify and biochemically characterize TBC1D7 as a stably associated and ubiquitous third core subunit of the TSC1-TSC2 complex. We demonstrate that the TSC1-TSC2-TBC1D7 (TSC-TBC) complex is the functional complex that senses specific cellular growth conditions and possesses Rheb-GAP activity.
TBC1D7 knockdown decreases the association of TSC1 and TSC2 leading to d

**Comment**: Answer contains complete unit instead of subunit, which was asked for.

In [33]:
WRONG_ANSWER_TYPE += 1
WITHIN_TOP5 += 1

In [34]:
print_prediction(incorrect_predictions[12], rev_vocab, with_context=True)

Id:
  5321bc309b2d7acc7e00000d
Question:
  what is the target protein of the drug idelalisib?
Answers:
 * PI3K-δ
 * phosphoinositol-3 kinase delta isoform
Predicted Answers:
 * ('PI3K', 0.99977916)
 * ('phosphoinositol-3', 0.83978611)
 * ('delta isoform', 0.78839111)
 * ('PI3Kδ', 0.57102162)
 * ('PI3Kδ (idelalisib)', 0.38207909)
 PI3Kδ (idelalisib). 
Idelalisib (PI3K inhibitor) 
In the last few years, several classes of potent and selective small molecule PI3K inhibitors have been developed, and at least fifteen compounds have progressed into clinical trials as new anticancer drugs. Among these, idelalisib
 idelalisib represents a first-in-class specific inhibitor of the phosphoinositol-3 kinase (PI3K) delta isoform.
PI3K inhibitor idelalisib 
the PI3K inhibitor idelalisib 
PI3K-δ inhibitor idelalisib
idelalisib (GS-110



**Comment**: Should count correct.

In [35]:
SHOULD_COUNT_CORRECT += 1

In [36]:
print_prediction(incorrect_predictions[13], rev_vocab, 10, with_context=True, context_char_limit=10000)

Id:
  532dcfc9d6d3ac6a34000021
Question:
  what is the characteristic domain of histone methyltransferases?
Answers:
 * SET domain
Predicted Answers:
 * ('SETDB1', 0.99537617)
 * ('SUV39H1', 0.92167568)
 * ('Trithorax', 0.82967973)
 * ('Polycomb repressive complex 2 (PRC2)', 0.76899356)
 * ('Meisetz', 0.74333292)
 * ('HMTase \nSET', 0.73406488)
 * ('subuni\nESC-E', 0.62423778)
 * ('C-terminal', 0.56639498)
 * ('WDR5', 0.51379132)
 * ('PRC2', 0.51079816)
C-terminal SET domain that catalyzes methylation of histone H3
not all members of the H3K4 methyltransferase family contain n-SET domains
protein methyltransferases (both protein arginine and lysine methyltransferases) and the relatedness of their catalytic domains. We identified 51 protein lysine methyltransferase proteins based on similarity to the canonical Drosophila Su(var)3-9, enhancer of zeste (E(z)), and trithorax (trx) domain
 a SET domain histone methyltransferase 
A common feature of the mammalian MLL/SET1 complexes is the pr

**Comment**: #1 should count correct ("ERG-associated protein with a SET domain, also called SETDB1"). However, it doesn't seem to have clearly understood that the answer type should be a domain.

In [37]:
SHOULD_COUNT_CORRECT += 1

In [38]:
print_prediction(incorrect_predictions[14], rev_vocab, 10)

Id:
  534427f8aeec6fbd07000009
Question:
  are most driver gene mutations synonymous or non-synonymous?
Answers:
 * non-synonymous
Predicted Answers:
 * ('SNVs', 0.91852707)
 * ('non-synonymous', 0.72141236)
 * ('ATRX', 0.71677607)
 * ('non', 0.59236729)
 * ('somatic', 0.5156346)
 * ('NRAS(Q61R)', 0.40278721)
 * ('neuroblastoma', 0.33723992)
 * ('MAPK', 0.32364196)
 * ('glutamate pathways', 0.2894012)
 * ('BRAF', 0.27417505)



**Comment**: Did not recognize multiple choice question. #1 is completely wrong. At least the correct answer is #2.

In [39]:
MULTPLE_CHOICE += 1
WITHIN_TOP5 += 1

In [40]:
print_prediction(incorrect_predictions[15], rev_vocab, with_context=True)

Id:
  534abe8aaeec6fbd07000013
Question:
  which heat shock protein is found to be upregulated during hsp90 inhibition?
Answers:
 * HSP70
Predicted Answers:
 * ('Akt phospho-rylation', 0.99945849)
 * ('Nox2', 0.99847507)
 * ('VEGF', 0.99395549)
 * ('Hsp70', 0.99362838)
 * ('5', 0.9898451)
HSP90 inhibition was associated with decreased neuroendocrine ErbB and IGF-I receptor expression, decreased Erk and Akt phospho-rylation and the induction of HSP70 expression.
Inhibition of Hsp90 upregulated the expression of Hsp70 and Hsp70-bound Nox2, 5 and promoted degradation.
Conversely, inhibition of HSP90 significantly increased the expression of both VEGF and HGF mRNA, and induced HSP70 protein in PHH cultures in vitro.



**Comment**: #1 Seems to be of the correct type, but the question asks for an increase, not decrease.

In [41]:
WITHIN_TOP5 += 1

In [42]:
print_prediction(incorrect_predictions[16], rev_vocab, with_context=True)

Id:
  54d907c84b1fd0d33c000008
Question:
  which molecule is targeted by a monoclonal antibody mepolizumab?
Answers:
 * interleukin-5
Predicted Answers:
 * ('imatinib mesylate', 0.96299136)
 * ('interleukin-5', 0.78903317)
 * ('mesylate', 0.69433331)
 * ('human interleukin-5', 0.53037494)
 * ('hyaluronan', 0.42750993)
Mepolizumab, a humanized monoclonal antibody that binds to and inactivates interleukin-5, has been shown to reduce asthma exacerbations in patients with severe eosinophilic asthma.
 Patients were assigned to receive mepolizumab, a humanized monoclonal antibody against interleukin-5, which was administered as either a 75-mg intravenous dose or a 100-mg subcutaneous dose, or placebo every 4 weeks for 32 weeks. 
Among developing therapies, biologics designed to block certain pro-inflammatory cytoki



**Comment**: #1 seems to be a medication, not a molecule.

In [43]:
WRONG_ANSWER_TYPE += 1
WITHIN_TOP5 += 1

In [44]:
print_prediction(incorrect_predictions[17], rev_vocab, 10, with_context=True)

Id:
  54e0ace81388e8454a000010
Question:
  is apobec3b protein predominantly cytoplasmic or nuclear?
Answers:
 *  nuclear
Predicted Answers:
 * ('APOBEC3F', 0.99415153)
 * ('A3B', 0.97979856)
 * ('APOBEC3G', 0.95807534)
 * ('APOBEC3D', 0.92119557)
 * ('HIV-1Δvif', 0.78807753)
 * ('APOBEC3B', 0.66793799)
 * ('L1', 0.66602021)
 * ('predominantly nuclear', 0.58152843)
 * ('cytosine deaminase', 0.22714922)
 * ('HIV-1', 0.15293275)
A3B is the only family member with steady-state nuclear localization
Here, we show that A3B nuclear import is an active process requiring at least one amino acid (Val54) within an N-terminal motif analogous to the nuclear localization determinant of the antibody gene diversification enzyme AID (activation-induced cytosine deaminase)
Our studies suggest that the present-day A3B enzyme retained the nuclear import mechanism of an ancestral AID protein during the expansion of the APOBEC3 locus in pr



**Comment**: Completely misunderstands Multiple Choice question.

In [45]:
MULTPLE_CHOICE += 1

In [46]:
print_prediction(incorrect_predictions[18], rev_vocab)

Id:
  54e0d7471388e8454a000015
Question:
  which molecule is targeted by a monoclonal antibody secukinumab?
Answers:
 * interleukin-17A
Predicted Answers:
 * ('IL-17A', 0.98593789)
 * ('Ustekinumab', 0.97535199)
 * ('Th-17A', 0.87675321)
 * ('IL-17RA', 0.68372774)
 * ('IL-17', 0.62270576)



**Comment**: Should count as correct

In [47]:
SHOULD_COUNT_CORRECT += 1

In [48]:
print_prediction(incorrect_predictions[19], rev_vocab, with_context=True, context_char_limit=1000)

Id:
  54fc91e96ad7dcbc12000001
Question:
  pbt2 has been tested for which disorder?
Answers:
 * Alzheimer's disease
Predicted Answers:
 * ('dementia', 0.94996768)
 * ('metal chaperones', 0.87829942)
 * ("mild Alzheimer's dementia", 0.83678919)
 * ('cognition', 0.83169907)
 * ('Metalloproteinase', 0.82517874)
PBT2 improved cognition in a phase II clinical trial with AD patients, and further clinical testing is currently underway.
In the second trial a successor compound, PBT2, was compared with placebo in 78 participants with mild Alzheimer's dementia; all were included in the intention-to-treat analysis. 
The second trial of PBT2 was more rigorously conducted and showed that after 12 weeks this compound appeared to be safe and well tolerated in people with mild Alzheimer's dementia. 
The purpose of this mini-review is to highlight the emerging notion that metal chaperones, such as PBT2 (Prana Biotechnology), modulate a variety of critical pathways affecting key aspects of the AD cascad

**Comment**: #1 correct. #2 is of wrong type.

In [49]:
SHOULD_COUNT_CORRECT += 1

In [50]:
print_prediction(incorrect_predictions[20], rev_vocab, with_context=True)

Id:
  54fc99f36ad7dcbc12000004
Question:
  neurostimulation of which nucleus is used for treatment of dystonia?
Answers:
 * globus pallidus internus
Predicted Answers:
 * ('VIM', 0.98844457)
 * ('globus pallidus', 0.95753789)
 * ('ventral intermediate thalamic nucleus', 0.93566036)
 * ('globus pallidus internus', 0.86834794)
 * ('globus pallidus internus (GPi)', 0.77401483)
Bilateral globus pallidus internus (GPi) DBS was performed in five SD patients and unilateral ventralis oralis anterior and posterior (Voa/Vop) nucleus of the thalamus DBS in two post-stroke hemiballismus patients. 
BACKGROUND: Deep brain stimulation of the internal pallidum (GPi-DBS) is effective for various types of drug-refractory primary dystonias. 
METHODS: In the parent trial, 40 patients were randomly assigned to either sham neurostimulation or neurostimulation of the internal globus pall



**Comment**: #1 is correct, see ideal answer:

"Neurostimulation of globus pallidus internus is effective for treatment of dystonia. Ventral intermediate thalamic nucleus has also been tested for neurostimulation in dystonia patients."

In [51]:
SHOULD_COUNT_CORRECT += 1

In [52]:
print_prediction(incorrect_predictions[21], rev_vocab, with_context=True)

Id:
  54ff45966ad7dcbc12000010
Question:
  which protein is causing netherton syndrome?
Answers:
 * LEKTI
 * lymphoepithelial Kazal type-related inhibitor
Predicted Answers:
 * ('SPINK5', 0.97955918)
 * ('Kazal-type inhibitor LEKTI-1 \nNetherton syndrome (NS)', 0.96757883)
 * ('serine protease', 0.94285423)
 * ('gene SPINK5 (serine protease inhibitor Kazal type 5) which encodes for a serine protease inhibitor LEKTI (lymphoepithelial Kazal type-related inhibitor)\nMutations in the serine protease inhibitor Kazal type 5 (SPINK5)', 0.89891535)
 * ('Kazal-type 5', 0.85271835)
Netherton syndrome (NTS) is a rare genetic skin disease caused by mutations in the serine protease inhibitor Kazal-type 5 gene, which encodes the lympho-epithelial Kazal-type-related inhibitor.
Netherton syndrome is caused by loss-of-function mutations in SPINK5 encoding the Kazal-type inhibitor LEKTI-1 
Netherton syndrome (NS) is a serious inherited skin disorder caused by mutations in the gene SPINK5 (serine proteas

**Comment**: Gene vs Protein type mismatch. Also, not how #1 goes across snippet boundaries.

In [53]:
WRONG_ANSWER_TYPE += 1

In [54]:
print_prediction(incorrect_predictions[22], rev_vocab, with_context=True)

Id:
  5505a587f73303d458000005
Question:
  what does mtor stands for?
Answers:
 * mammalian target of rapamycin
Predicted Answers:
 * ('rapamycin', 0.99975544)
 * ('mammalian target of rapamycin', 0.81508398)
 * ('Rapamycin (mTOR)\n mammalian target of rapamycin', 0.69096982)
 * ('', 0.34639639)
 * ('mammalian Target', 0.28946447)
 mammalian target of rapamycin (mTOR)
mammalian target of rapamycin (mTOR)
mammalian target or rapamycin (mTOR)
mammalian target of rapamycin (mTOR) 
 mammal target of rapamycin (mTOR)
mammalian Target of Rapamycin (mTOR)
 mammalian target of rapamycin (mTOR) 



**Comment**: Linguistically a good answer, doesn't know how abbriviations work.

In [55]:
WITH_TOP5 += 1

NameError: name 'WITH_TOP5' is not defined

In [ ]:
print_prediction(incorrect_predictions[23], rev_vocab, with_context=True)

**Comment**: Accross Snippet Boundaries, #2 should count correct

In [ ]:
WITHIN_TOP5 += 1

In [ ]:
print_prediction(incorrect_predictions[24], rev_vocab, 10, with_context=True)

**Comment**: Wrong answer type: enzymatic activity vs Protein.

In [ ]:
WRONG_ANSWER_TYPE += 1

In [ ]:
print_prediction(incorrect_predictions[25], rev_vocab)

**Comment**: Almost Correct (missing closing parenthesis)

In [ ]:
SOMEWHAT_CORRECT += 1

In [ ]:
print_prediction(incorrect_predictions[26], rev_vocab, with_context=True, context_char_limit=100000)

**Comment**: #1 likely correct. From Context:

"Rrm3p is a 5'-to-3' DNA helicase that helps replication forks traverse protein-DNA complexes. Its absence leads to increased fork stalling and breakage at over 1,000 specific sites located throughout the Saccharomyces cerevisiae genome"

In [ ]:
SOMEWHAT_CORRECT += 1

# Summary

Total Predictions:

In [ ]:
len(predictions)

Correct or should count correct:

In [ ]:
(len(predictions) - len(incorrect_predictions)) + SHOULD_COUNT_CORRECT

Somewhat correct:

In [ ]:
SOMEWHAT_CORRECT

Clearly incorrect:

In [ ]:
len(incorrect_predictions) - SHOULD_COUNT_CORRECT - SOMEWHAT_CORRECT

Only somewhat correct or incorrect, but correct answer is within top5:

In [ ]:
WITHIN_TOP5

Top Answer Type is wrong:

In [ ]:
WRONG_ANSWER_TYPE

System failed to recognize multiple choice questions:

In [ ]:
MULTPLE_CHOICE

# 